In [ ]:
import socket
import numpy as np

import sys
import subprocess

def execute(fname):
    """
    # https://www.semicolonworld.com/question/59697/return-value-from-one-python-script-to-another
    s2_out = subprocess.check_output([sys.executable, fname])
    return s2_out
    """
    
    # https://github.com/mynameisvinn/piegrad/blob/master/PieGrad.py
    # https://stackoverflow.com/questions/16877323/getting-return-information-from-another-python-script
    # https://stackoverflow.com/questions/6357361/alternative-to-execfile-in-python-3
    namespace={}  # environment
    with open(fname, "rb") as source_file:
        code = compile(source_file.read(), fname, "exec")
    exec(code, namespace)  # put results in namespace env 
    
    # send numpy results as bytes https://markhneedham.com/blog/2018/04/07/python-serialize-deserialize-numpy-2d-arrays/
    # could picklet
    res = namespace['res']
    res_bytes = res.tobytes()
    return res_bytes
    

def receive_file(c, out_fname):
    """receive file from client and save as out_fname.
    
    https://stackoverflow.com/questions/9382045/send-a-file-through-sockets-in-python
    """
    # data = c.recv(1024).decode('utf-8')

    l = c.recv(1024)
    print("reading", l)
    f = open(out_fname,'wb')
    f.write(l)
    f.close()
    return True

"""
spin up server for remote execution. 

https://www.geeksforgeeks.org/socket-programming-python/
"""

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
endpoint = ("localhost", 9510)
s.bind(endpoint)
s.listen(5)
while True:

    # connect with whoever pinged server
    c, addr = s.accept()      
    print('connected to', addr )

    # receive source file from client
    temp_fname = "barfoo.py"
    _ = receive_file(c, temp_fname)

    # server evaluates source and returns results
    res = execute(temp_fname)  # res is numpy answer in bytes
    print(np.frombuffer(res))

    # return result back to client
    c.send(res)  # res is sent as bytes, so it will have to be converted back to numpy array

    # close connection 
    c.close() 

connected to ('127.0.0.1', 51127)
reading b'# testsdfs\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b)\n\nres = matmul(a, b) * 100'
[124.16616022 106.92021143 111.29285124 181.69292153  49.6375236
  45.17678305  62.16320099  83.63875997 165.14777875  79.57262827
 144.39317821 159.31858539  81.43877072  67.16250328  78.7139585
 107.13891234]
connected to ('127.0.0.1', 51144)
reading b'# testsdfs\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b)\n\nres = matmul(a, b) * 100'
[ 95.33076608 105.7275287   95.72129289 120.89962649  15.8051894
  52.86839501  44.33995595  20.67152534  88.14003367 105.77704131
  90.13188399 118.8074465   41.20922231  39.57755553  47.03860474
  48.56421315]
connected to ('127.0.0.1', 51259)
reading b'\n\nimport numpy as np\n\na = np.random.rand(4, 4)\nb = np.random.rand(4, 4)\n\ndef matmul(a, b):\n    return a.dot(b)\n\nres = m